In [1]:
from scipy.optimize import fsolve
import statsmodels.formula.api as sm
import statsmodels.regression.linear_model as sm
import statsmodels.api as sm
import pandas as pd
import pickle
import time
import datetime
import numpy as np
import statistics
import statsmodels.api as sm
import math
import pandas as pd
from scipy.stats import norm
from scipy import stats
from scipy.stats import ttest_1samp

In [37]:
edf = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\edf_final_0121.pkl")
edf.dropna(inplace = True)

edf["year"] = edf["fyear"]+1
edf.reset_index(inplace = True,drop = True)

edf["ret-1"] = (edf["prc"]/edf["-20"])-1
edf["ret-3"] = (edf["prc"]/edf["-60"])-1
edf["ret-6"] = (edf["prc"]/edf["-120"])-1
edf["ret-12"] = (edf["prc"]/edf["-250"])-1
edf["booklev"] = edf["lt"]/edf["at"]
edf["invprice"] = 1/edf["prc"]

In [38]:
# 由於有6個變數 故一間公司至少需要7筆資料

gvk_list = np.unique(edf["gvk"].tolist()).tolist()
short_gvk = []
for i in range(0,len(gvk_list)):
    gv = gvk_list[i]
    t1 = edf[edf["gvk"]==gv]
    if len(t1) < 7 :
        short_gvk.append(gv)
    else:
        pass
        
for i in range(0,len(short_gvk)):
    gvp = short_gvk[i]
    tt = edf[edf["gvk"]==gvp].index
    edf.drop(tt,inplace = True)
edf.reset_index(inplace = True,drop = True)
edf

,gvk,fyear,csho,lt,std,prc,ret,mv,sigma_year,at,...,invprice,dltt,cl,l,sigma_v,dd,e+f/f,edf,year,booklev
0,001004,1984.0,6.036,68.020,0.017761,19.0,-0.084337,114.684,0.280818,155.405,...,0.052632,16.415,51.605,59.8125,0.225764,4.256063,2.917392,0.00001,1985.0,0.437695
1,001004,1985.0,9.099,102.906,0.021837,24.0,0.246753,218.376,0.345273,198.287,...,0.041667,25.022,77.884,90.3950,0.2841,5.050362,3.415797,0.0,1986.0,0.518975
2,001004,1986.0,10.536,100.781,0.02021,32.125,0.259804,338.469,0.319544,235.091,...,0.031128,12.477,88.304,94.5425,0.278135,6.266164,4.580072,0.0,1987.0,0.428689
3,001004,1987.0,15.870,134.451,0.032184,27.75,-0.174721,440.3925,0.508877,284.948,...,0.036036,23.781,110.670,122.5605,0.436672,2.872934,4.593266,0.002033,1988.0,0.471844
4,001004,1988.0,16.009,184.811,0.017972,29.125,0.104265,466.262125,0.28416,356.391,...,0.034335,25.707,159.104,171.9575,0.24021,5.773488,3.711496,0.0,1989.0,0.518562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16146,212782,1999.0,210.210,3728.292,0.021803,26.1875,0.321767,5504.874375,0.344732,5752.383,...,0.038186,1617.879,2110.413,2919.3525,0.272461,4.93428,2.885649,0.0,2000.0,0.64813
16147,212782,2000.0,210.000,3278.932,0.020317,23.65,-0.099048,4966.49992,0.321242,5978.953,...,0.042283,1610.559,1668.373,2473.6525,0.257762,3.758997,3.00776,0.000085,2001.0,0.548412
16148,212782,2001.0,210.000,3879.033,0.023272,14.83,-0.375842,3114.299984,0.367967,6516.01,...,0.067431,2164.537,1714.496,2796.7645,0.261049,1.29649,2.113537,0.097403,2002.0,0.595308
16149,212782,2002.0,210.000,3950.248,0.041159,16.440001,0.092359,3452.400112,0.650785,6779.949,...,0.060827,2234.491,1715.757,2833.0025,0.453326,1.734951,2.218636,0.041375,2003.0,0.582637


In [39]:
# famamacbeth第一階段-時間序列-每間公司單獨做回歸

gvk_list = np.unique(edf["gvk"].tolist()).tolist()
da = [[0,0,0,0,0,0,0,0]]
data = pd.DataFrame(da,columns=[0,1,2,3,4,5,6,7])
for i in range(0,len(gvk_list)):
    
    beta = []
    gv = gvk_list[i]
    t1 = edf[edf["gvk"]==gv]
    if t1.empty:
        pass
    else:
        yd = t1["icc"].tolist()
        t1.reset_index(inplace = True,drop = True)
        ttt = t1.loc[:,["drf","ln_a","mtb","ret-1","booklev","stdy"]]

        tnp = ttt.values   
        tnp = sm.add_constant(tnp)

        lm = sm.OLS(yd,tnp.astype(float)).fit()
        if lm.rsquared ==1:
            pass
        else:
            
            beta.append(gv)
            beta.append(lm.params[0])
            beta.append(lm.params[1])
            beta.append(lm.params[2])
            beta.append(lm.params[3])
            beta.append(lm.params[4])
            beta.append(lm.params[5])
            beta.append(lm.params[6])

            bbeta = [beta]
            data2 =  pd.DataFrame(bbeta,columns=[0,1,2,3,4,5,6,7])
            data = pd.concat([data,data2],axis=0)

data.rename(columns={0:"gvkey",1:"const",2:"drf",3:"ln_a",4:"mtb",5:"ret-1",6:"booklev",7:"stdy"},inplace = True)

data_step_1 = data.iloc[1:,:]
data_step_1

,gvkey,const,drf,ln_a,mtb,ret-1,booklev,stdy
0,001004,-0.323572,-0.000093,0.033882,-0.009161,-0.096262,0.359229,0.012607
0,001034,0.084264,-0.000092,-0.019825,-0.033534,0.081901,0.248951,0.105122
0,001038,0.137021,0.000308,0.023555,-0.036042,-0.113174,-0.223588,-0.033062
0,001054,0.054234,0.000025,0.006546,-0.007889,0.109001,-0.044387,0.050648
0,001055,1.195747,-0.005126,-0.065277,-0.227878,-0.623804,-0.130415,-0.056593
...,...,...,...,...,...,...,...,...
0,063185,-0.029802,0.000461,0.013760,-0.004454,-0.093401,0.180530,-0.004732
0,063621,-0.388996,0.001141,0.097730,0.001552,0.008816,-0.424453,0.011614
0,063643,0.055998,0.002407,0.002999,-0.000130,0.023612,-0.079788,0.007362
0,063927,-0.583305,0.000203,0.182339,0.021233,0.076311,-0.882203,0.008632


In [40]:
# famamacbeth第二階段 對係數做回歸

ga = [[0,0,0,0,0,0,0,0]]
gata = pd.DataFrame(ga,columns=[0,1,2,3,4,5,6,7])

for i in range(1982,2006):

    geta = []
    r_gamma = []
    beta_all_gamma = []
    t1 = edf[edf["year"]==i]
    if t1.empty:
        pass
    elif len(t1) < 8:
        pass
    else:
        gvk_gamma = t1["gvk"].tolist()
        
        for j in range(0,len(gvk_gamma)):
            # print(j)
            beta_gamma = []
            gv = gvk_gamma[j]
            if data_step_1[data_step_1["gvkey"]==gv].empty:
                pass
            else:
                r_gamma.append(edf[(edf["gvk"]==gv)&(edf["year"]==i)]["icc"].values[0])
                # beta_gamma.append(data_step_1[data_step_1["gvkey"]==gv]["const"].values[0])
                beta_gamma.append(data_step_1[data_step_1["gvkey"]==gv]["drf"].values[0])
                beta_gamma.append(data_step_1[data_step_1["gvkey"]==gv]["ln_a"].values[0])
                beta_gamma.append(data_step_1[data_step_1["gvkey"]==gv]["mtb"].values[0])
                beta_gamma.append(edf[(edf["gvk"]==gv)&(edf["year"]==i)]["ret-1"].values[0])
                beta_gamma.append(edf[(edf["gvk"]==gv)&(edf["year"]==i)]["booklev"].values[0])
                beta_gamma.append(edf[(edf["gvk"]==gv)&(edf["year"]==i)]["stdy"].values[0])               

                beta_all_gamma.append(beta_gamma)
        y = r_gamma
        x = np.array(beta_all_gamma)
        x = sm.add_constant(x)
        lm = sm.OLS(y,x.astype(float)).fit()
        if lm.rsquared ==1:
            pass
        
        else:
            geta.append(i)
            geta.append(lm.params[0])
            geta.append(lm.params[1])
            geta.append(lm.params[2])
            geta.append(lm.params[3])
            geta.append(lm.params[4])
            geta.append(lm.params[5])
            geta.append(lm.params[6])

            ggeta = [geta]
            gata2 =  pd.DataFrame(ggeta,columns=[0,1,2,3,4,5,6,7])
            gata = pd.concat([gata,gata2],axis=0)

gata.rename(columns={0:"year",1:"const",2:"drf",3:"ln_a",4:"mtb",5:"ret-1",6:"booklev",7:"stdy"},inplace = True)
# gata.rename(columns={0:"year",1:"const",2:"drf",3:"ln_a",4:"mtb",5:"ret-1",6:"booklev",7:"stdy"},inplace = True)
# gata.rename(columns={0:"year",1:"const",2:"drf",3:"ln_a",4:"mtb",5:"ret-1",6:"booklev"},inplace = True)
# gata.rename(columns={0:"year",1:"const",2:"drf",3:"ln_a",4:"mtb",5:"ret-1"},inplace = True)
data_step_2 = gata.iloc[1:,:]
data_step_2

,year,const,drf,ln_a,mtb,ret-1,booklev,stdy
0,1982,-0.045217,4.439553,-0.128723,-0.391126,-0.105365,0.076821,0.044974
0,1983,-0.044666,2.187604,-0.072018,-0.153680,-0.019458,0.053337,0.011348
0,1984,-0.040019,5.243599,-0.085443,-0.196981,-0.004968,0.040497,0.079173
0,1985,-0.035275,2.045295,-0.058855,-0.139281,-0.006989,0.021804,0.096274
0,1986,-0.009109,1.435274,-0.063397,-0.085504,-0.016365,0.007836,0.061476
0,1987,-0.018563,2.949364,-0.058732,-0.088885,-0.021757,0.023419,0.041391
0,1988,-0.051367,2.717136,-0.055996,-0.133160,0.010808,0.026614,0.081398
0,1989,-0.023702,1.761767,-0.068436,-0.089031,-0.037861,0.007818,0.086263
0,1990,-0.030220,-0.499908,-0.043960,-0.094657,-0.023076,0.019945,0.097199
0,1991,-0.012859,-0.260353,-0.093522,-0.111520,-0.031721,0.036442,0.064953


In [42]:
feta = []
feta.append(data_step_2["const"].mean())
feta.append(data_step_2["drf"].mean())
feta.append(data_step_2["ln_a"].mean())
feta.append(data_step_2["mtb"].mean())
feta.append(data_step_2["ret-1"].mean())
feta.append(data_step_2["booklev"].mean())
feta.append(data_step_2["stdy"].mean())
# feta.append(data_step_2["invprice"].mean())
fata =  pd.DataFrame(feta)

feta2 = []
feta2.append(ttest_1samp(data_step_2["const"], 0))
feta2.append(ttest_1samp(data_step_2["drf"], 0))
feta2.append(ttest_1samp(data_step_2["ln_a"], 0))
feta2.append(ttest_1samp(data_step_2["mtb"], 0))
feta2.append(ttest_1samp(data_step_2["ret-1"], 0))
feta2.append(ttest_1samp(data_step_2["booklev"], 0))
feta2.append(ttest_1samp(data_step_2["stdy"], 0))
# feta2.append(ttest_1samp(data_step_2["invprice"], 0))
fata2 =  pd.DataFrame(feta2)
fata3 = pd.concat([fata,fata2],axis = 1)

fata3.rename(columns={0: 'mean'}, inplace=True)
fata3.rename({0: 'constant',1:"drf",2:"ln_a",3:"mtb",4:"ret-1",5:"booklev",6:"stdy"}, inplace=True)

In [43]:
# 如下表所示 drf顯著正相關 代表破產風險越高的公司 確實存在越高的預期報酬
fata3

,mean,statistic,pvalue
constant,-0.008869,-1.635383,1.155822e-01
drf,1.282338,3.140413,4.585883e-03
ln_a,-0.029688,-2.637806,1.470739e-02
mtb,-0.118200,-5.009175,4.559453e-05
ret-1,-0.020896,-3.885404,7.474355e-04
booklev,0.027741,8.240663,2.569820e-08
stdy,0.069370,11.463393,5.476587e-11
